## Data Ingestion

In [167]:
from datetime import datetime
from datetime import date
import time
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# pandas options
pd.options.display.float_format = '{:,.2f}'.format

# USE CAUTION/UNCOMMENT if necessary
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 150)

# plotting
%matplotlib inline

# Constants:
SAND_PRICE_IN_USD = 7.23
SAND_PRICE_DATETIME = date.today()

# Initial dataframe is the top-level, need to flatten
df_initial = pd.read_json('top_wallets_and_transactions_with_USD_prices.json')
display(df_initial.head())
print(df_initial.shape)
print(df_initial.columns)

datetime.date(2021, 11, 29)

,holders
0,"{'address': '0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35', 'balance': 1108241744.15, 'share': 36.94, 'transactions': [{'timestamp': 1630991473, 'tr..."
1,"{'address': '0x5f5b8942ae6325227d5d82e5759e837271fa2a67', 'balance': 522999999.99999994, 'share': 17.43, 'transactions': [{'timestamp': 1628783320..."
2,"{'address': '0x8ffa64fb50559c3ff09a1022b84b2c5233ed8068', 'balance': 281512136.62667745, 'share': 9.38, 'transactions': [{'timestamp': 1625488642,..."
3,"{'address': '0xf977814e90da44bfa03b6295a0616a897441acec', 'balance': 261899174.6018669, 'share': 8.73, 'transactions': [{'timestamp': 1636830622, ..."
4,"{'address': '0x07d2601739709c25fe0afd50ec961ba589311caa', 'balance': 195000000.0, 'share': 6.5, 'transactions': [{'timestamp': 1628789705, 'transa..."


(200, 1)
Index(['holders'], dtype='object')


In [120]:
def flat_sand_data():
    '''Flatten JSON object and normalize to transaction level data.'''
    with open('top_wallets_and_transactions_with_USD_prices.json','r') as json_file:
        json_data = json.load(json_file)
        df = pd.json_normalize(
            json_data,
            record_path=["holders","transactions"],
            sep="_",
            meta=[
                     ['holders','address'],
                     ['holders','balance'],
                     ['holders','share'],
            ],
            errors='ignore'
        )
        df = df.rename(
            columns={
                'type': 'tx_type',
                'from': 'address_from',
                'to': 'address_to',
                'value': 'abs_value'
            }
        )
        
        df = df.reindex(
            columns = [
                'holders_address',
                'holders_balance', 
                'holders_share',
                'transactionHash',
                'timestamp',
                'tx_type',
                'address_from',
                'address_to',
                'abs_value',
                'USD_price_at_timestamp',
            ]
        )
        
    return df

df_flat = flat_sand_data()
df_flat.to_csv('df_flat.csv', header=True) # Useful to view DF as CSV.
# df_flat.head()


In [163]:
# df_tx takes a copy of df_flat and makes for better readability.

df_tx = df_flat.copy()

df_tx['uniq_transID'] = df_tx[['holders_address','timestamp']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df_tx['date_time'] = pd.to_datetime(df_tx['timestamp'], unit='s') # Convert from UNIX timestamp.
df_tx['value'] = np.where(df_tx['holders_address']!=df_tx['address_from'], -df_tx['abs_value'], df_tx['abs_value']) # !!!Positive values are sales, Negative values indicate purchase.
df_tx['value_in_USD'] = df_tx['value'] * df_tx['USD_price_at_timestamp'] # Cash Flow data in USD.

# df_tx = df_tx.rename(columns={'timestamp': 'date_time'})    
df_tx.to_csv('df_tx.csv', header=True) # Useful to view DF as CSV.
df_tx.columns


Index(['holders_address', 'holders_balance', 'holders_share',
       'transactionHash', 'timestamp', 'tx_type', 'address_from', 'address_to',
       'abs_value', 'USD_price_at_timestamp', 'uniq_transID', 'date_time',
       'value', 'value_in_USD'],
      dtype='object')

In [157]:
# Need to sanity check. 

# df_tx.loc[df_tx["holders_address"] =="0xd6216fc19db775df9774a6e33526131da7d19a2c"] # all positive(sales) values.
df_tx.loc[df_tx["holders_address"] =="0x07d2601739709c25fe0afd50ec961ba589311caa",]

,holders_address,holders_balance,holders_share,transactionHash,timestamp,tx_type,address_from,address_to,abs_value,USD_price_at_timestamp,uniq_transID,date_time,value,value_in_USD
293,0x07d2601739709c25fe0afd50ec961ba589311caa,"195,000,000.00",6.50,0x995dbd713aff6054c07c87d56385ee52546924fffb8664b2a0a799d9702bc3ce,1628789705,transfer,0x07d2601739709c25fe0afd50ec961ba589311caa,0xd27275c3ffe7fb48a3802abcc43cb5938cd5a5fa,"3,000,000.00",0.65,0x07d2601739709c25fe0afd50ec961ba589311caa_1628789705,2021-08-12 17:35:05,"3,000,000.00","1,960,078.58"
294,0x07d2601739709c25fe0afd50ec961ba589311caa,"195,000,000.00",6.50,0x640908bc8f699c8e42b287feed04cfb05393dade15b99402f4c8bde571c90aa3,1628785457,transfer,0x07d2601739709c25fe0afd50ec961ba589311caa,0x3cc634fffe9fec2fdf0d4d969722daea16a6b3d8,"3,000,000.00",0.65,0x07d2601739709c25fe0afd50ec961ba589311caa_1628785457,2021-08-12 16:24:17,"3,000,000.00","1,960,078.58"
295,0x07d2601739709c25fe0afd50ec961ba589311caa,"195,000,000.00",6.50,0x13c09d9c43a16c92438f309a32601cd16be88aa6ec0d97b8c3c48a195db24800,1628785220,transfer,0x07d2601739709c25fe0afd50ec961ba589311caa,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"9,000,000.00",0.65,0x07d2601739709c25fe0afd50ec961ba589311caa_1628785220,2021-08-12 16:20:20,"9,000,000.00","5,880,235.74"
296,0x07d2601739709c25fe0afd50ec961ba589311caa,"195,000,000.00",6.50,0x820addd0a703fd20f9b3bc531a91bc9abdd4af8d1c507eb2418de60143de868e,1597772543,transfer,0x5f5b8942ae6325227d5d82e5759e837271fa2a67,0x07d2601739709c25fe0afd50ec961ba589311caa,"210,000,000.00",0.05,0x07d2601739709c25fe0afd50ec961ba589311caa_1597772543,2020-08-18 17:42:23,"-210,000,000.00","-11,021,789.53"


### Add Current Value of Balance (WIP)

In [ ]:
for wallet in set(df_tx['holders_address']):
    df_tx.append(
        {
            'holders_address': str(wallet),
            'holders_balance': ,
            'holders_share': ,
            'transactionHash': np.NaN,
            'timestamp': SAND_PRICE_DATETIME, 
            'tx_type': 'end_value', 
            'address_from': np.NaN, 
            'address_to': np.NaN,
            'abs_value': , 
            'USD_price_at_timestamp': SAND_PRICE_IN_USD, 
            'uniq_transID': np.NaN, 
            'date_time': SAND_PRICE_DATETIME,
            'value':, 
            'value_in_USD':
        }
    )

## Wallet-Level Data:

In [155]:
# wallet_level_data (df) uses df_flat to run min method on UNIX timestamp vs date-time.

wallet_level_data = pd.DataFrame(
    df_flat[
            ["holders_address",
             "timestamp",
        ]
    ]
    .groupby(["holders_address"])
    .min(numeric_only=True)
)

wallet_level_data.insert(loc=0, column='row_num', value=np.arange(len(wallet_level_data)))
wallet_level_data['row_num'] = wallet_level_data.reset_index(inplace=True)
wallet_level_data.drop(['row_num'], axis=1, inplace=True)
wallet_level_data['uniq_transID'] = wallet_level_data[['holders_address','timestamp']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

wallet_level_data = wallet_level_data.rename(
            columns={
                'transactionHash': 'initial_tx_hash',
                'timestamp': 'initial_date'
            }
        )

wallet_level_data = pd.merge(wallet_level_data, df_tx, on="uniq_transID",how='inner')
# wallet_level_data['holding_period_month'] = (pd.Timestamp(date.today()) - wallet_level_data['date_time']).astype('timedelta64[M]')
wallet_level_data['holding_period_days'] = (time.time() - wallet_level_data['timestamp'])/86400

# wallet_level_data.rename(
#             columns={
#                 'holders_balance_x': 'holders_balance',
#                 'holders_share_x': 'holders_share',
#                 'value': 'abs_value'
#             }
#         )
# wallet_level_data.reindex(
#             columns = [
#                 'holders_balance',
#                 'holders_share', 
#                 'holders_address',
#                 'initial_tx_hash',
#                 'initial_date',
#                 'holding_period_month',
#                 'tx_type',
#                 'address_from',
#                 'address_to',
#                 'abs_value',
#                 'value',
#                 'USD_price_at_timestamp',
#                 'value_in_USD'
#                 'holding_period_month'
#             ]
#         )

display(wallet_level_data)
wallet_level_data.to_csv('wallet_level_data.csv', header=True) # Useful to view DF as CSV.

,holders_address_x,initial_date,uniq_transID,holders_address_y,holders_balance,holders_share,transactionHash,timestamp,tx_type,address_from,address_to,abs_value,USD_price_at_timestamp,date_time,value,value_in_USD,holding_period_days
0,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a,1634408375,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a_1634408375,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a,"279,251.22",0.01,0x84a3d0a06046d96dd2969d288d0f0a752e992fe040a83312c48f2dfe855024a3,1634408375,transfer,0x9d68c9dfbeca4a0d2a35365f01b90e8626af4417,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a,"279,251.22",0.74,2021-10-16 18:19:35,"-279,251.22","-207,564.59",44.13
1,0x0211f3cedbef3143223d3acf0e589747933e8527,1637331123,0x0211f3cedbef3143223d3acf0e589747933e8527_1637331123,0x0211f3cedbef3143223d3acf0e589747933e8527,"604,495.41",0.02,0x013899c638efe612b6e0ce09ce71014fe1c92448368899848f51c790bbc678a2,1637331123,transfer,0x579d30bd4ea860c40698b4c2a85bf99e21ad6e1a,0x0211f3cedbef3143223d3acf0e589747933e8527,"2,908.28",4.35,2021-11-19 14:12:03,"-2,908.28","-12,639.11",10.30
2,0x0404d96d2583799bc73edb8d230aa037fc22b379,1626340280,0x0404d96d2583799bc73edb8d230aa037fc22b379_1626340280,0x0404d96d2583799bc73edb8d230aa037fc22b379,"110,480,542.03",3.68,0xb35f10d5e29b34c4a662a61f2db01c564b7aacaf0a003359d164acaa2aa5be59,1626340280,transfer,0x0404d96d2583799bc73edb8d230aa037fc22b379,0x0b9d0312347fac94baa511cd150789a34791c2ca,"3,000,000.00",0.61,2021-07-15 09:11:20,"3,000,000.00","1,833,634.38",137.51
3,0x07445065963c2d563cd70ddf2aa49fc771e59a98,1637562190,0x07445065963c2d563cd70ddf2aa49fc771e59a98_1637562190,0x07445065963c2d563cd70ddf2aa49fc771e59a98,"2,205,375.00",0.07,0x210750b99b9ee23a36730b24e8fa5f07572f16fd6ea9e2ed45b304c0b983ea2c,1637562190,transfer,0xd9d8d445f4e578ffb68349ff49186d6edc97b4f1,0x07445065963c2d563cd70ddf2aa49fc771e59a98,"1,770,000.00",3.92,2021-11-22 06:23:10,"-1,770,000.00","-6,943,437.70",7.63
4,0x07d2601739709c25fe0afd50ec961ba589311caa,1597772543,0x07d2601739709c25fe0afd50ec961ba589311caa_1597772543,0x07d2601739709c25fe0afd50ec961ba589311caa,"195,000,000.00",6.50,0x820addd0a703fd20f9b3bc531a91bc9abdd4af8d1c507eb2418de60143de868e,1597772543,transfer,0x5f5b8942ae6325227d5d82e5759e837271fa2a67,0x07d2601739709c25fe0afd50ec961ba589311caa,"210,000,000.00",0.05,2020-08-18 17:42:23,"-210,000,000.00","-11,021,789.53",468.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,1637748491,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98_1637748491,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,"1,646,379.81",0.05,0xcdcb8cff430ffa78a0920b34815b83bffe3bb25bf8b2dde05ab3ec8c48e8d936,1637748491,transfer,0x90c78b4170815533f3bd4e7206ca3af8ce231ef5,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,93.74,6.96,2021-11-24 10:08:11,-93.74,-652.14,5.47
246,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e84,1633491240,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e84_1633491240,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e84,"170,000.00",0.01,0x44cff1197ec759c1cdbe6374df1baf23218e53923770f643af3476cc5e9370ce,1633491240,transfer,0xa171579cdb365092db1379ebedde2318ad352a05,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e84,"1,000.00",0.81,2021-10-06 03:34:00,"-1,000.00",-808.77,54.75
247,0xfed694dc2ce345dc23a5a915cdb5d55b258351c9,1633415163,0xfed694dc2ce345dc23a5a915cdb5d55b258351c9_1633415163,0xfed694dc2ce345dc23a5a915cdb5d55b258351c9,"1,048,005.46",0.03,0x1e116401484f567fba2411c3504659e2beb79705e880bd39653e37cb6379cdf2,1633415163,transfer,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0xfed694dc2ce345dc23a5a915cdb5d55b258351c9,47.00,0.81,2021-10-05 06:26:03,-47.00,-38.08,55.63
248,0xff0cb0351a356ad16987e5809a8daaaf34f5adbe,1625458618,0xff0cb0351a356ad16987e5809a8daaaf34f5adbe_1625458618,0xff0cb0351a356ad16987e5809a8daaaf34f5adbe,"854,380.67",0.03,0xd3a5e4bbab1b81b75f47b94380aa1b593afd02fc757d97fb1200f99b45d58897,1625458618,transfer,0x74de5d4fcbf63e00296fd95d33236b9794016631,0xff0cb0351a356ad16987e5809a8daaaf34f5adbe,"411,646.43",0.27,2021-07-05 04:16:5

# Sanity Checks

In [158]:
print(df_tx.columns)

# Only pulls in transaction type: transfer.
print(f'\n\nThe type of transaction pulled:')
display(df_tx.value_counts(['tx_type']))

# Date Range, seems reasonable
print(f'\n\nThe first date of date range is:')
display(df_tx['date_time'].min())
print(f'\n\nThe last date of date range is:')
display(df_tx['date_time'].max())

# Any same to/from addresses?
print(f'\n\nWere there transactions where the tokens were transfer from one wallet to itself?')
display(df_tx.value_counts(df['address_from']==df['address_to']))
                       
# 18k transactions in line with Ethan's estimates.
print(f'\n\nThe total number of transactions in the dataset:')
print(f'Shape: {df_tx.shape}')


Index(['holders_address', 'holders_balance', 'holders_share',
       'transactionHash', 'timestamp', 'tx_type', 'address_from', 'address_to',
       'abs_value', 'USD_price_at_timestamp', 'uniq_transID', 'date_time',
       'value', 'value_in_USD'],
      dtype='object')


The type of transaction pulled:


tx_type 
transfer    18139
dtype: int64



The first date of date range is:


Timestamp('2020-08-05 10:27:29')



The last date of date range is:


Timestamp('2021-11-24 20:38:01')



Were there transactions where the tokens were transfer from one wallet to itself?


False    18139
dtype: int64



The total number of transactions in the dataset:
Shape: (18139, 14)


In [159]:
# The sum of the holders_share should be less than 100%
print(f'\n\nThe sum of the holders_share is:')
total_share = df_tx[['holders_address', 'holders_share']].groupby('holders_address').max().sort_values('holders_share', ascending=False)
display(total_share['holders_share'].sum())


# The sum of the balance should be less than 100% of the total # of circulating tokens
print(f'\n\nThe sum of the holders_balance is:')
total_balance = df_tx[['holders_address', 'holders_balance']].groupby('holders_address').max().sort_values('holders_balance', ascending=False)
display(total_balance['holders_balance'].sum())


# The USD price at each date_time stamp
print(f'\n\nThe USD price at each date_time stamp:')
display(df_tx[['date_time', 'USD_price_at_timestamp']].sort_values('date_time'))


# Count NaN or empty values
print(f'\n\nThe count of NaN or empty values:')
display(df_tx.isnull().sum())


# Why were these value_in_USD NaN or empty? Some of the timestamps are in 2021
print(f'\n\nThe rows containing NaN or empty values:')
display(df_tx.loc[df['value_in_USD'].isnull()])

# df_null.to_csv('export_dataframe.csv', index = False, header=True) # Useful to open DF as CSV.




The sum of the holders_share is:


97.41000000000001



The sum of the holders_balance is:


2916123141.7947025



The USD price at each date_time stamp:


,date_time,USD_price_at_timestamp
28,2020-08-05 10:27:29,NaN
27,2020-08-11 23:08:29,NaN
4284,2020-08-11 23:08:29,NaN
26,2020-08-12 11:01:48,NaN
6768,2020-08-13 20:43:14,NaN
...,...,...
4610,2021-11-24 20:36:55,7.37
5365,2021-11-24 20:36:55,7.37
4608,2021-11-24 20:37:14,7.46
15356,2021-11-24 20:37:40,7.46




The count of NaN or empty values:


holders_address            0
holders_balance            0
holders_share              0
transactionHash            0
timestamp                  0
                          ..
USD_price_at_timestamp    11
uniq_transID               0
date_time                  0
value                      0
value_in_USD              11
Length: 14, dtype: int64



The rows containing NaN or empty values:


,holders_address,holders_balance,holders_share,transactionHash,timestamp,tx_type,address_from,address_to,abs_value,USD_price_at_timestamp,uniq_transID,date_time,value,value_in_USD
25,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x186374b502fbe1e1f7842c6f603579ce221ae674f8be9e907c39d3c7a85ca4d4,1597351394,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"50,000,000.00",NaN,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35_1597351394,2020-08-13 20:43:14,"50,000,000.00",NaN
26,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x29f57014c89f919ef944f930b130b29e78811af14c8ad8254eae866548b97dd8,1597230108,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x7dab328cf8c8cf7567a96e3e74ebd5aff2a0ae3d,"360,000,000.00",NaN,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35_1597230108,2020-08-12 11:01:48,"360,000,000.00",NaN
27,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0xc25d4d40dac5e0b8051fa3f82276908de3fc41d9ecf10887edf6f5eb3e22d74d,1597187309,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0xb701dec4186c8c7f1a7047b33f9f9a39febb1db1,"6,000,000.00",NaN,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35_1597187309,2020-08-11 23:08:29,"6,000,000.00",NaN
28,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x9563be30c250b56dbb0917300b80c220e422625269afaa91dbb281d4ed3a9dfd,1596623249,transfer,0xeaa0993e1d21c2103e4f172a20d29371fbaf6d06,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"3,000,000,000.00",NaN,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35_1596623249,2020-08-05 10:27:29,"-3,000,000,000.00",NaN
1710,0x08f14cfcc8f0b67bfac70d3f93623775577e0909,"21,149,927.87",0.70,0xb495559d52737757eddcc91fe0de8a51fab7aef09af27494b5f0c322941da8f6,1637124356,transfer,0x002d91c86d026319ba0d159c84ffea03bc00adf9,0x08f14cfcc8f0b67bfac70d3f93623775577e0909,"76,185.00",NaN,0x08f14cfcc8f0b67bfac70d3f93623775577e0909_1637124356,2021-11-17 04:45:56,"-76,185.00",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6768,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"1,500,000.00",0.05,0x186374b502fbe1e1f7842c6f603579ce221ae674f8be9e907c39d3c7a85ca4d4,1597351394,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"50,000,000.00",NaN,0x5366dc49e76ba0173921c636aa23dcd00e4b296b_1597351394,2020-08-13 20:43:14,"-50,000,000.00",NaN
12648,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,"258,050.28",0.01,0x25590d0792a4008266676a5c71c64d083301f3e702e5ce653c5927f921282dd9,1637339263,transfer,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,0x08a2103a11e898ec31709fff6883d1658e20071d,453.97,NaN,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06_1637339263,2021-11-19 16:27:43,453.97,NaN
12926,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,"258,050.28",0.01,0xdf591ace8f377d1d05b8cb71e30ea1d36b06ff759ea71d74ce6c101f441c00bf,1635415733,transfer,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,0x3507e4978e0eb83315d20df86ca0b976c0e40ccb,"57,145.00",NaN,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06_1635415733,2021-10-28 10:08:53,"57,145.00",NaN
17057,0x4da0eda15a164e78fddc0eddbee9295a78aa230e,"167,151.02",0.01,0xd8b0e5d7bf5194dbb7536f742e82a80d1510609b7dc81fdcce0777d900e3531e,1637339469,transfer,0x3dd49f67e9d5bc4c5e6634b3f70bfd9dc1b6bd74,0x4da0eda15a164e78fddc0eddbee9295a78aa230e,"8,236.09",NaN,0x4da0eda15a164e78fddc0eddbee9295a78aa230e_1637339469,2021-11-19 16:31:09,"-8,236.09",NaN


## Holding Period Checks:

In [18]:
wallet_level_data['holding_period_month'].value_counts(bins = [0,1, 2, 3,6,12,16])

(-0.001, 1.0]    139
(6.0, 12.0]       18
(1.0, 2.0]        16
(2.0, 3.0]        15
(3.0, 6.0]         9
(12.0, 16.0]       6
Name: holding_period_month, dtype: int64

## Net Cash Flow by Wallet

In [19]:
df.groupby(['holders_address']).sum().sort_values('value_in_USD')


,abs_value,value,USD_price_at_timestamp,value_in_USD
holders_address,,,,
0xf977814e90da44bfa03b6295a0616a897441acec,"245,528,132.04","-44,884,670.50",18.74,"-116,508,068.50"
0x8d6f396d210d385033b348bcae9e4f9ea4e045bd,"28,360,000.00","-28,360,000.00",11.67,"-91,885,437.26"
0x08f14cfcc8f0b67bfac70d3f93623775577e0909,"302,820,614.32","-21,100,594.32","2,382.00","-45,845,967.88"
0x326d9f47ba49bbaac279172634827483af70a601,"60,000,005.00","-10,600,005.00",64.10,"-44,975,392.27"
0x37d493e32ca5ef914389c8cb2e33e81805309d96,"60,016,739.35","-9,003,697.59",43.80,"-37,552,865.47"
...,...,...,...,...
0xdd10c6e0b95ea74a6e3fcb1b7f119681e83f2dd8,"12,400,000.00","-2,500,000.00",47.57,"15,642,017.98"
0x3f148612315aae2514ac630d6faf0d94b8cd8e33,"13,304,376.41","3,708,819.55",388.00,"16,069,236.60"
0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,"8,617,165.50","2,683,369.36",749.19,"21,980,762.08"


In [109]:
df.groupby(['holders_address']).sum().sort_values('value')

,abs_value,value,USD_price_at_timestamp,value_in_USD
holders_address,,,,
0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"4,838,710,063.85","-1,161,289,936.15",7.35,"160,665,045.12"
0x5f5b8942ae6325227d5d82e5759e837271fa2a67,"1,037,000,000.00","-523,000,000.00",2.13,"-2,347,350.65"
0x8ffa64fb50559c3ff09a1022b84b2c5233ed8068,"481,447,009.37","-281,512,136.63",18.50,"-1,519,098.94"
0x07d2601739709c25fe0afd50ec961ba589311caa,"225,000,000.00","-195,000,000.00",2.01,"-1,221,396.64"
0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"516,306,912.49","-51,500,000.00",27.09,"-6,502,122.94"
...,...,...,...,...
0x2434e7e28b2b139511bc2b6f31e14ee1e6e03b4f,"9,714,833.56","986,118.36",215.98,"4,583,658.84"
0xc1ff7cea93d2324a76e1cb19c98d5e0a3a2b6b3f,"75,010,717.88","2,210,717.88","3,259.98","4,918,093.10"
0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,"8,617,165.50","2,683,369.36",749.19,"21,980,762.08"
